1. 문서의 내용을 읽는다

In [ ]:
%pip install python-docx

In [ ]:
from docx import Document

document = Document('./tax.docx')

print(f'document == {dir(document)}')

full_text = ''

for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph =={paragraph.text}')
    full_text += f'{paragraph.text}\n'

2. 문서를 쪼갠다

In [ ]:
%pip install tiktoken

In [29]:
import tiktoken 

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
    
    return text_list

In [ ]:
chunk_list = split_text(full_text, 1500)

In [ ]:
3. 문서 임베딩

In [ ]:
%pip install chromadb

In [ ]:
import chromadb

chroma_client = chromadb.Client()

In [ ]:
collection_name = "tax_collection"
tax_collection = chroma_client.create_collection(name=collection_name)

In [ ]:
from langchain_ollama import OllamaEmbeddings
import os

embeddings = OllamaEmbeddings(model=os.getenv("embedmodel"))

In [50]:
tax_collection = chroma_client.get_or_create_collection(name="tax_collection")

In [51]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [53]:
tax_collection.add(documents=chunk_list, ids=id_list)

4. 유사도 검색

In [56]:
query = '연봉 7천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc = tax_collection.query(query_texts=query, n_results=3)


In [ ]:
retrieved_doc['documents'][0]

In [ ]:
5. LLM 질의

In [62]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
import os

llm = ChatOllama(model=os.getenv("model"))

response = ChatPromptTemplate.from_messages(
    [
        {
            "role": "system",
            "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}",
        },
        {"role": "user", "content": query},
    ]
)

In [ ]:
response.messages[0]